In [1]:
#@title Import relevant modules
import os
import sys
import pandas as pd
import numpy as np
import geopandas as gpd
import sqlalchemy as sq
from matplotlib import pyplot as plt
from IPython.display import clear_output
from dotenv import load_dotenv
from DataService import DataService
from ClimateDataRequester import ClimateDataRequester as cdr

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_215844/1399629943.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (callin

In [2]:
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)


In [3]:
os.chdir('/data')
load_dotenv('docker/.env')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')

In [4]:
def push_data(df: pd.DataFrame) -> None:
    dataService = DataService(PGDB, PGUSER, PGPW)
    db_con = dataService.connect()
    df.to_sql("WeatherData", db_con, if_exists="append", index=False)
    dataService.cleanup()


In [5]:
def dataProcessA(df: pd.DataFrame, stationID: str) -> None:
    dataService = DataService(PGDB, PGUSER, PGPW)
    db_con = dataService.connect()
    try:
        df.drop(columns=['Data Quality', 'Max Temp Flag', 'Mean Temp Flag', 'Min Temp Flag', 'Heat Deg Days Flag', 'Cool Deg Days Flag', 'Spd of Max Gust (km/h)',
                         'Total Rain Flag', 'Total Snow Flag', 'Total Precip Flag', 'Snow on Grnd Flag', 'Dir of Max Gust Flag', 'Spd of Max Gust Flag',
                         'Heat Deg Days (°C)', 'Cool Deg Days (°C)', 'Longitude (x)', 'Latitude (y)', 'Station Name', 'Dir of Max Gust (10s deg)'], inplace=True)
    except:
        df.to_csv("data/failed/" + str(df.iloc[0, 0]) + "_unexpected_column_names.csv", index=False)

    # Climate ID	Date/Time	Year	Month	Day	Max Temp (Â°C)	Min Temp (Â°C)	Mean Temp (Â°C)	Total Rain (mm)	Total Snow (cm)	Total Precip (mm)	Snow on Grnd (cm)	Dir of Max Gust (10s deg)	Spd of Max Gust (km/h)
    # ClimateID Date Year Month Day MaxTemp MinTemp MeanTemp TotalRain TotalSnow TotalPrecip SnowOnGrnd DirOfMaxGust SpdOfMaxGust
    df.rename(columns={df.columns[0]: "ClimateID"}, inplace=True)
    df.rename(columns={df.columns[1]: "Date"}, inplace=True)
    df.rename(columns={df.columns[2]: "Year"}, inplace=True)
    df.rename(columns={df.columns[3]: "Month"}, inplace=True)
    df.rename(columns={df.columns[4]: "Day"}, inplace=True)
    df.rename(columns={df.columns[5]: "MaxTemp"}, inplace=True)
    df.rename(columns={df.columns[6]: "MinTemp"}, inplace=True)
    df.rename(columns={df.columns[7]: "MeanTemp"}, inplace=True)
    df.rename(columns={df.columns[8]: "TotalRain"}, inplace=True)
    df.rename(columns={df.columns[9]: "TotalSnow"}, inplace=True)
    df.rename(columns={df.columns[10]: "TotalPrecip"}, inplace=True)
    df.rename(columns={df.columns[11]: "SnowOnGrnd"}, inplace=True)

    df.dropna(subset=['MeanTemp'], inplace=True)
    df.loc[df['SnowOnGrnd'].isnull(), 'SnowOnGrnd'] = 0
    df.loc[df['TotalRain'].isnull(), 'TotalRain'] = 0
    df.loc[df['TotalSnow'].isnull(), 'TotalSnow'] = 0
    df.loc[df['TotalPrecip'].isnull(), 'TotalPrecip'] = 0
    df['MaxTemp'] = np.where(df['MaxTemp'].isnull(),
                             df['MeanTemp'], df['MaxTemp'])
    df['MinTemp'] = np.where(df['MinTemp'].isnull(),
                             df['MeanTemp'], df['MinTemp'])

    df[['ClimateID', 'Date']] = df[['ClimateID', 'Date']].astype(str)
    df[['Year', 'Month', 'Day']] = df[['Year', 'Month', 'Day']].astype(int)
    df[['MaxTemp', 'MinTemp', 'MeanTemp', 'TotalRain', 'TotalSnow', 'TotalPrecip', 'SnowOnGrnd']] = df[[
        'MaxTemp', 'MinTemp', 'MeanTemp', 'TotalRain', 'TotalSnow', 'TotalPrecip', 'SnowOnGrnd']].astype(float)

    # we try a db push, but if it fails, we place the data in a csv file
    # try:
    push_data(df)
    query = sq.text("UPDATE public.\"StationsDly\" SET \"scraped\" = True WHERE \"Climate ID\" like CAST(\'{}\' AS TEXT);".format(stationID))
    db_con.execute(query)
    # except:
    #     df.to_csv("Failed/" + str(df.iloc[0, 0]) +
    #             "_data_failed_dbpush.csv", index=False)
    dataService.cleanup()


In [6]:
tableName = "public.\"StationsDly\""
query = "SELECT * FROM " + tableName + ";"


dataService = DataService(PGDB, PGUSER, PGPW)
db_con = dataService.connect()
dfStations = gpd.GeoDataFrame.from_postgis(query, db_con, geom_col='geometry')


In [7]:
# get list of MANITOBA stations
mbStations = dfStations.loc[dfStations['Province'] == 'MANITOBA']

# remove stations with NaN DLY First Year
mbStations = mbStations.loc[mbStations['DLY First Year'].notnull()]

# reindex 
mbStations.reset_index(drop=True, inplace=True)

In [8]:
# try first 2 stations
# mbStations = mbStations.iloc[0:3, :]

In [9]:
mbStations.count()

,Station Name,Province,Latitude,Longitude,Elevation,Climate ID,WMO Identifier,TC Identifier,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,CRnum,scraped
0,BACHELORS ISLAND MARINE,MANITOBA,51.8,-99.9,255.9,5040131,71143.0,WBL,1994,2012,1994.0,2012.0,1994.0,2012.0,1996.0,2004.0,POINT (5652086.326 1767996.578),0,None
1,BERENS RIVER A,MANITOBA,52.4,-97.0,222.2,5030204,NaN,YBV,2015,2022,2015.0,2022.0,2018.0,2022.0,NaN,NaN,POINT (5853400.266 1814545.117),0,None
2,BERENS RIVER CS,MANITOBA,52.4,-97.0,221.9,5030203,71158.0,WCF,1985,2022,1985.0,2022.0,1985.0,2022.0,1985.0,2007.0,POINT (5853400.266 1814545.117),0,None


In [10]:
requester = cdr()
df = pd.DataFrame()

for index, row in mbStations.iterrows():
                
    stationID = str(row['Climate ID'])
    errQuery = sq.text("UPDATE public.\"StationsDly\" SET \"scraped\" = False WHERE \"Climate ID\" like CAST(\'{}\' AS TEXT);".format(stationID))
    if row['scraped'] == False or row['scraped'] == None:
        try:
            startYr = 1995
            endYr = 2022
            if row['DLY First Year'] != np.NAN or row['DLY Last Year'] != np.NAN or row['DLY Last Year'] > 1995:
                if row['DLY First Year'] > 1995:
                    startYr = row['DLY First Year']
                if row['DLY Last Year'] > 1995:
                    endYr = row['DLY Last Year']
                df = requester.get_data("MB", stationID, startYr, endYr)
                clear_output(wait=False)

                print(df.head())
                
                if not df.empty:
                    dataProcessA(df, stationID)
                else:
                    db_con.execute(errQuery)
        except Exception as e:
            print("Failed to scrape " + stationID)
            print(e)
            db_con.execute(errQuery)
    else:
        print("Data for station " + str(stationID) + " already exists.")

    print("Processed row " + str(index + 1) + " of " + str(len(mbStations)))


   Longitude (x)  Latitude (y)     Station Name  Climate ID   Date/Time  Year   
0          -97.0          52.4  BERENS RIVER CS     5030203  2022-12-01  2022  \
1          -97.0          52.4  BERENS RIVER CS     5030203  2022-12-02  2022   
2          -97.0          52.4  BERENS RIVER CS     5030203  2022-12-03  2022   
3          -97.0          52.4  BERENS RIVER CS     5030203  2022-12-04  2022   
4          -97.0          52.4  BERENS RIVER CS     5030203  2022-12-05  2022   

   Month  Day  Data Quality  Max Temp (°C) Max Temp Flag  Min Temp (°C)   
0     12    1           NaN          -13.2           NaN          -23.5  \
1     12    2           NaN          -14.4           NaN          -27.8   
2     12    3           NaN          -16.0           NaN          -22.8   
3     12    4           NaN           -8.8           NaN          -20.7   
4     12    5           NaN          -18.2           NaN          -32.1   

  Min Temp Flag  Mean Temp (°C) Mean Temp Flag  Heat Deg Days 

In [11]:

dataService.cleanup()